# Rossmann-Store-Sales data preparation

In [ ]:
from datetime import datetime
import pandas as pd
from pandas.plotting import scatter_matrix
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn import preprocessing

sales_train_file="../data/rossmann-store-sales/train.csv"
sales_test_file="../data/rossmann-store-sales/test.csv"
stores_file="../data/rossmann-store-sales/store.csv"

In [ ]:
# read in files
df_sales=pd.read_csv(sales_train_file)
display(df_sales)
df_stores=pd.read_csv(stores_file)
print(df_stores.info())
display(df_stores)

## Ajust Date Types

In [ ]:
df_stores.rename(columns={"PromoInterval": "Promo2Interval"}, inplace=True)
df_sales["Date"]=pd.to_datetime(df_sales["Date"])
df_sales["StateHoliday"]=df_sales["StateHoliday"].astype('str')
df_stores["Promo2"]=df_stores['Promo2'].astype('bool')

## Join Tables

In [ ]:
df_joined = pd.merge(df_sales, df_stores, how="inner", on="Store")
display(df_joined)

## Null Value handling

In [ ]:
# problem location 
df_joined.isna().mean()

In [ ]:
# drop rows with NULL CompetionDistance
df_joined = df_joined.dropna(subset=["CompetitionDistance"])
# fill CompetitionOpenSinceMonth
#z.loc[:, 'C'] = z.A + 2

df_joined.loc[:, "CompetitionOpenSinceMonth"] = df_joined.apply(lambda x: x["Date"].month if (pd.isna(x["CompetitionOpenSinceMonth"])) else x["CompetitionOpenSinceMonth"], axis=1)
df_joined.loc[:, "CompetitionOpenSinceYear"] = df_joined.apply(lambda x: x["Date"].year if (pd.isna(x["CompetitionOpenSinceYear"])) else x["CompetitionOpenSinceYear"], axis=1)
## change data types
df_joined= df_joined.astype({"CompetitionOpenSinceMonth": int, "CompetitionOpenSinceYear": int}, copy=False)
#df_joined.loc[:, "CompetitionOpenSinceMonth"] = df_joined.loc[:, "CompetitionOpenSinceMonth"].astype(int)
#df_joined.loc[:, "CompetitionOpenSinceYear"] = df_joined.loc[:, "CompetitionOpenSinceYear"].astype(int)

In [ ]:
# handel promo2
# locate the problem
print ("Value Count",  df_joined["Promo2"].value_counts())
print("Number of lines without promo2: ", df_joined[(df_joined["Promo2"] == False) & (df_joined["Promo2SinceWeek"].isna())].shape[0])
print("Number of lines with promo2 and Null Values in Promo2 science: ", df_joined[(df_joined["Promo2"] == True) & (df_joined["Promo2SinceWeek"].isna())].shape[0])

# Set all to 0 when no Promo2
df_joined.fillna({"Promo2Interval":0}, inplace=True)
df_joined.fillna({"Promo2SinceWeek":0}, inplace=True)
df_joined.fillna({"Promo2SinceYear":0}, inplace=True)

#df_joined.loc[:,['Promo2Interval', 'Promo2SinceWeek', 'Promo2SinceYear']].fillna(value=0, inplace=True)

# change data types
df_joined= df_joined.astype({"Promo2SinceWeek": int, "Promo2SinceYear": int}, copy=False)
#df_joined.loc[:, "Promo2SinceWeek"] = df_joined.loc[:, "Promo2SinceWeek"].astype(int)
#df_joined.loc[:, "Promo2SinceYear"] = df_joined.loc[:, "Promo2SinceYear"].astype(int)
#df_joined.dtypes

In [ ]:
num_attributes = df_joined.select_dtypes(include=["int64", "float64"])
cat_attributes = df_joined.select_dtypes(include=["object", "bool"])
# num values per cat attribute
cat_attributes.apply(lambda x: x.unique().shape[0])

## Split Attributes / Create new ones

In [ ]:
def calc_month_between(start_date : datetime, end_date : datetime) -> datetime: 
    return (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)


df_joined.loc[:, "Year"]=df_joined.loc[:, "Date"].dt.year.astype(int)
df_joined.loc[:, "Month"]=df_joined.loc[:, "Date"].dt.month.astype(int)
df_joined.loc[:, "Day"]=df_joined.loc[:, "Date"].dt.day.astype(int)
df_joined.loc[:, "DayName"]=df_joined.loc[:, "Date"].dt.day_name().astype(str) 
df_joined
df_joined.loc[:, "CompetitionMonth"] = df_joined.apply(lambda x: calc_month_between(datetime(month=x["CompetitionOpenSinceMonth"],year=x["CompetitionOpenSinceYear"],day=1 ),datetime.today()), axis=1)
#datetime(month=x["CompetitionOpenSinceMonth"],year=x["CompetitionOpenSinceYear"],day=1)


## Show correlations

In [ ]:

%matplotlib inline
df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].hist(bins=50, figsize=(20,15))
plt.show()

#df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].plot(x="DayOfWeek", y="Sales").line()
df_sales_on_day_agg = df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].groupby(["DayOfWeek"]).mean()
df_sales_on_day_agg.rename(columns= {"Sales" : "Avg"},inplace=True)
df_sales_on_day_agg["Count"] = df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].groupby(["DayOfWeek"]).count()
df_sales_on_day_agg["Max"] = df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].groupby(["DayOfWeek"]).max()
df_sales_on_day_agg["Min"] = df_joined.loc[df_joined["Sales"]>0,["Sales","DayOfWeek"]].groupby(["DayOfWeek"]).min()

display (df_sales_on_day_agg)

df_sales_on_day_agg["Avg"].plot.box()

plt.show()

In [ ]:
%matplotlib inline
aux1 = df_joined.loc[(df_joined["Sales"]>0) & (df_joined["CompetitionDistance"]>0),["CompetitionDistance", "Sales"]].groupby("CompetitionDistance").sum().reset_index()
aux1.plot.scatter(x="CompetitionDistance", y="Sales", color="DarkBlue", label="Sales/Distance")
plt.show()
display(aux1)

## Binning CompetitionDistance

In [ ]:
est = preprocessing.KBinsDiscretizer(n_bins=25, encode='ordinal',subsample=None).fit(df_joined[["CompetitionDistance"]])
df_joined["CompetitionDistanceBin"] = est.transform(df_joined[["CompetitionDistance"]])    
display(df_joined)

In [ ]:
# calc mean for each bin and assign it to each instance
mean_dist = df_joined[["CompetitionDistance","CompetitionDistanceBin"]].groupby("CompetitionDistanceBin").mean().reset_index()
display(mean_dist)
df_joined = pd.merge(df_joined, mean_dist, how="inner", on="CompetitionDistanceBin")
display(df_joined)